# Problema: Camino del coste mínimo
Un senderista estáa planifcando una ruta de montaña haciendo uso de un plano de coordenadas nxm. En cada
posición (i, j) de dicho plano se representa, mediante un núumero natural, el grado de dificultad de visitar esa casilla.
Cuanto más elevado es dicho valor más difícil es acceder a esa posición. Por ejemplo:

![alt text](imagen.png "Matriz de Pesos")

El objetivo es obtener el camino de coste mínimo desde la casilla origen (0,0) hasta la casilla (n-1,m-1) asumiendo que solo son validos tres tipos de movimiento partiendo de una casilla arbitraria (i,j):

1. Derecha: (i,j+1)
2. Abajo: (i+1,j)
3. Abajo y Derecha (Diagonal): (i+1,j+1)

### Inicializar las librerías necesarias

In [8]:
import numpy as np

### Cargar mapaa

In [27]:
with open('100.map') as f:
    lines = [line.rstrip() for line in f]

mapa=[]
for i in range(1,len(lines)):
    mapa.append(list(map(int,lines[i].split(" "))))
mapa=np.array(mapa)

## Solución por Fuerza Bruta



In [6]:
def naive(mapa,row,col):
  P1,P2,P3=np.inf,np.inf,np.inf #Establecer a infinitos los valores de los futuros caminos a priori
  if row==mapa.shape[0]-1 and col==mapa.shape[1]-1: #Caso Base: El camino ha llegado a la casilla final desde la casilla inicio
    return mapa[row][col]
  else:
    if row<mapa.shape[0]-1: #Comprobar que puedo moverme una casilla hacia abajo
      P1=mapa[row][col]+naive(mapa,row+1,col) #Almacenar el valor de la casilla actual y del camino hacia el que me muevo
    if col<mapa.shape[1]-1: #Comprobar que puedo moverme a una casilla a la derecha
      P2=mapa[row][col]+naive(mapa,row,col+1)
    if row<mapa.shape[0]-1 and col<mapa.shape[1]-1: #Comprobar que puedo moverme una casilla en diagonal hacia abajo y derecha
      P3=mapa[row][col]+naive(mapa,row+1,col+1)
    return min(P1,min(P2,P3)) #Devolver el camino minimo de los explorados

In [11]:
%time print(f"Solución Camino Mínimo (Naive): {naive(mapa,0,0)}")

Solución Camino Mínimo (Naive): 13
CPU times: total: 15.6 ms
Wall time: 15 ms


Para mapas de orden superior a 10x10 la solucion por fuerza bruta no es eficiente en tiempo.

## Solución voraz

Esta solución solo es ideal cuando el tamaño del problema es grande o complejo. Además es posible que esta solución no encuentre la solución óptima sino una solución subóptima. Por otro lado, al ser una solución recursiva también puede ocurrir que se haga overflow de la pila de llamadas. Para este problema en concreto, tomando en cuenta como maneja la memoria Python es una solución ineficiente y por tanto no combiene su uso para tamaños de problema grandes.

In [28]:
def greedy(mapa,row,col):
  if row==mapa.shape[0]-1 and col==mapa.shape[1]-1: #Caso Base: El camino ha llegado a la casilla final desde la casilla inicio
    return mapa[row][col]
  if row==mapa.shape[0]-1: #Si he llegado a la ultima fila continuar hacia la derecha
    return mapa[row][col]+greedy(mapa,row,col+1)
  elif col==mapa.shape[1]-1: #Si he llegado a la ultima columna continuar hacia la abajo
    return mapa[row][col]+greedy(mapa,row+1,col)
  else: #Comprobar todos los caminos para todas las direcciones disponibles
    return mapa[row][col]+min(greedy(mapa,row+1,col+1),min(greedy(mapa,row+1,col),greedy(mapa,row,col+1))) #Devolver el coste de la posicion actual y el cam,ino minimo restante

In [29]:
%time print(f"Solución Camino Mínimo (Voraz): {greedy(mapa,0,0)}")

KeyboardInterrupt: 

## Solución con Programación Dinámica

### Aplicando Recursividad

In [14]:
store=np.full(mapa.shape, -1, dtype=int)
def recursive_Store(mapa,store,row,col): #Programación Dinámica con Almacen Recursivo
  P1,P2,P3=np.inf,np.inf,np.inf #Establecer a infinitos los valores de los furturos caminos a priori
  if store[row][col]!=-1: #Comprobar si se ha almacenado el coste de algun camino completo en el almacen
    return store[row][col]
  if row==mapa.shape[0]-1 and col==mapa.shape[1]-1: #Caso Base: Si se ha llegado a la casilla final almacenar el coste del camino y devolverlo
    store[row][col]==mapa[row][col]
    return mapa[row][col]
  else:
    if row<mapa.shape[0]-1: #Comprobar que puedo moverme una casilla hacia abajo
      P1=mapa[row][col]+recursive_Store(mapa,store,row+1,col) #Almacenar el valor de la casilla actual y del camino hacia el que me muevo
    if col<mapa.shape[1]-1: #Comprobar que puedo moverme a una casilla a la derecha
      P2=mapa[row][col]+recursive_Store(mapa,store,row,col+1)
    if row<mapa.shape[0]-1 and col<mapa.shape[1]-1: #Comprobar que puedo moverme una casilla en diagonal hacia abajo y derecha
      P3=mapa[row][col]+recursive_Store(mapa,store,row+1,col+1)
    store[row][col]=min(P1,min(P2,P3)) #Almacenar el camino minimo actual de los explorados
    return min(P1,min(P2,P3)) #Devolver el camino minimo de los explorados

In [15]:
%time print(f"Solución Camino Mínimo (Almacen Recursivo): {recursive_Store(mapa,store,0,0)}")

Solución Camino Mínimo (Almacen Recursivo): 13
CPU times: total: 0 ns
Wall time: 525 µs


Esta solución es eficiente en tiempo para mapaas de orden menor a 500x500, a partir de este tamaño se agota la pila de llamadas

### Solución Iterativa 

#### Almacen sin optimización

In [16]:
def iterative_Store(mapa):
  store=np.full(mapa.shape, -1, dtype=int) #Inicializar el Almacen
  store[0,:]=np.cumsum(mapa[0,:]) #Almacenar el coste de seguir un camino vertical hacia abajo 
  store[:,0]=np.cumsum(mapa[:,0]) #Almacenar el coste de seguir un camino horizontal hacia la derecha
  for i in range(1,mapa.shape[0]):
    for j in range(1,mapa.shape[1]):
      store[i][j]=min(mapa[i][j]+store[i-1][j-1],min(mapa[i][j]+store[i][j-1],mapa[i][j]+store[i-1][j])) #Almacenar los costes minimos posibles desde la posicion actual y las direcciones posibles siguientes
  return store[-1][-1]

In [17]:
%time print(f"Solución Camino Mínimo (Almacen Iterativo): {iterative_Store(mapa)}")

Solución Camino Mínimo (Almacen Iterativo): 13
CPU times: total: 0 ns
Wall time: 998 µs


#### Almacen con Optimizacion
Existe una forma de optimizar el almacen usando tan solo dos vectores del tamaño de las filas del mapaa que se iran actualizando conforme avance el algoritmo, reduciendo el consumo de memoria y por tanto acelerando el algoritmo. No obstante para este problema en ocasiones puede no encontrar la solución óptima

In [18]:
def iterative_Op_Store(mapa):
  store_0=np.full(mapa.shape[1], -1, dtype=int)
  store_1=np.full(mapa.shape[1], -1, dtype=int)
  store_1=np.cumsum(mapa[:,0]) #Inicializar el Vector 2 con la suma acumulativa del vector anterior
  for i in range(mapa.shape[0]):
    store_0=store_1.copy() #Actualizar el primer vector (Hacer una copia para que los punteros no se modifiquen unos a otros)
    store_1=mapa[i].copy() #Actualizar el segundo vector
    store_1[0]+=store_0[0] #Acumular el coste del primer movimiento del camino actual
    for j in range(1,mapa.shape[1]):
      store_1[j]+=min(store_1[j-1],min(store_0[j],store_0[j-1])) #Actualizar el coste con el camino minimo de la posicion actual y las direcciones posibles
  return store_1[-1]


In [19]:
%time print(f"Solución Camino Mínimo (Almacen Iterativo Optimizado): {iterative_Op_Store(mapa)}")

Solución Camino Mínimo (Almacen Iterativo Optimizado): 14
CPU times: total: 0 ns
Wall time: 999 µs


## Solución con Backtracking y Ramificación y Poda

En las soluciones anteriores solo se permitían 3 movimientos:
1.   Movimiento hacia abajo
2.   Movimiento hacia la derecha
3.   Movimiento en diagonal hacia abajo y derecha

Sin embargo, las restricciones del problema van a ser actualizadas, ahora se puede realizar cualquier movimiento hacia una celda adyacente, habilitando 8 movimientos posibles.

Para hacer esta solución más eficiente, se realizará una poda del arbol de caminos posibles empleando una solución pesimista, que será una solución subóptima al problema resuelta por otra técnica de las vistas anteriormente (usando las restricciones de 3 movimientos), en este caso, se empleará programación dinámica iterativa. 

In [20]:
def pessimistic_solution(mapa): #Solución pesimista basada en programación dinámica
  store=np.full(mapa.shape, -1, dtype=int) #Inicializar el Almacen
  store[0,:]=np.cumsum(mapa[0,:]) #Almacenar el coste de seguir un camino vertical hacia abajo 
  store[:,0]=np.cumsum(mapa[:,0]) #Almacenar el coste de seguir un camino horizontal hacia la derecha
  for i in range(1,mapa.shape[0]):
    for j in range(1,mapa.shape[1]):
      store[i][j]=min(mapa[i][j]+store[i-1][j-1],min(mapa[i][j]+store[i][j-1],mapa[i][j]+store[i-1][j])) #Almacenar los costes minimos posibles desde la posicion actual y las direcciones posibles siguientes
  return store[-1][-1]

#### Backtracking

In [21]:
def backtracking(mapa,row,col,best,acum,store,solution):
  #print(f"Row: {row} Col: {col}")
  if row==mapa.shape[0]-1 and col==mapa.shape[1]-1: #Caso Base: Llegar a la casilla final desde la inicial
    best=min(acum,best) #Actualizar el mejor coste de los caminos con el coste del camino actual
    store[row][col]=acum
    solution.append(best)
    return
  else:
    for i in range(-1,2):
        for j in range(-1,2):
          if not (i==0 and j==0):
            if row+i>=0 and row+i<mapa.shape[0] and col+j>=0 and col+j<mapa.shape[1]:
              acum+=mapa[row+i][col+j]
              if acum+max(mapa.shape[0]-1-(row+i),mapa.shape[1]-1-(col+i))<=best and acum < store[row+i][col+j]:
                store[row+i][col+j]=acum
                backtracking(mapa,row+i,col+j,best,acum,store,solution)
              acum-=mapa[row+i][col+j]

In [22]:
best=pessimistic_solution(mapa)
print(f"Solución Pesimista: {best}")
store=np.full(mapa.shape, np.inf)
solution=list()
%time backtracking(mapa,0,0,best,mapa[0][0],store,solution)
print(f"Solución Camino Mínimo (Backtracking): {solution[-1]}")

Solución Pesimista: 13
CPU times: total: 0 ns
Wall time: 997 µs
Solución Camino Mínimo (Backtracking): 12


#### Ramificación y Poda

In [23]:
from queue import PriorityQueue
def branchBound(mapa,best,store):
  p_queue = PriorityQueue()
  acum=mapa[0][0]
  p_queue.put((acum,0,0)) #Encolar el primer nodo
  store[0][0]=acum
  while not p_queue.empty():
    acum,row,col=p_queue.get()
    if row==mapa.shape[0]-1 and col==mapa.shape[1]-1:
      best=min(best,acum)
    if acum+max(mapa.shape[0]-1-row,mapa.shape[1]-1-col)<best:
      for i in range(-1,2):
        for j in range(-1,2):
          if not (i==0 and j==0):
            if row+i>=0 and row+i<mapa.shape[0] and col+j>=0 and col+j<mapa.shape[1] and acum+mapa[row+i][col+j]<store[row+i][col+j]:
              store[row+i][col+j]=acum+mapa[row+i][col+j]
              p_queue.put((acum+mapa[row+i][col+j],row+i,col+j))
  return best


In [26]:
best=pessimistic_solution(mapa)
print(f"Solución Pesimista: {best}")
store=np.full(mapa.shape,best)
%time print(f"Solución Camino Mínimo (Ramificacion y Poda): {branchBound(mapa,best,store)}")

Solución Pesimista: 3080
Solución Camino Mínimo (Ramificacion y Poda): 2917
CPU times: total: 20 s
Wall time: 20.5 s
